# Data analysis

To run the analysis without dependency issues, please use Python 3.11.3.

### 1. Install libraries:


In [ ]:
!pip3 install pandas==1.5.3
!pip3 install xgboost==3.0.1
!pip3 install scikit-learn==1.2.2
!pip3 install shap==0.47.2
!pip3 install matplotlib==3.7.1
!pip3 install tables==3.8.0
!pip3 install gensim==4.3.0
!pip3 install numba==0.57.0

### 2. Load libraries:

In [ ]:
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shap
import xgboost as xgb
from scipy.stats import randint, uniform
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

In [ ]:
# Load the model:

best_model_2 = joblib.load('output/best_model_2.pkl')
best_params = best_model_2.get_params() # Save the best parameters.

### 3. Load the data:

In [ ]:
data = pd.read_csv("/Users/justina/Desktop/Data Science/retirement_data_all.csv")

In [ ]:
print(data.dtypes.value_counts()) # 3 columns are numerical, and upon further inspection, some of them are not needed.
num_columns = data.select_dtypes(include=['float64'])
print(num_columns)

# remove these columns that are not needed:
data = data.drop(columns=["exrate", "ex009age"])

### 4. XGBoost
#### Relevant links:

| Note                         | Link                                        |
|------------------------------|----------------------------------------------------|
| Working with categorical variables |[Tutorial](https://xgboost.readthedocs.io/en/latest/tutorials/categorical.html) |
| Information about XGBRegressor and possible parameters |[Documentation](https://xgboost.readthedocs.io/en/latest/python/python_api.html#)     |
| Hyperparameter tuning - RandomizedSearchCV |[Tutorial](https://dev.to/uche_4rm_germany/grid-and-randomized-hyperparameter-optimization-for-xgboost-algorithms-159k) |


#### 4.1 Prepare the data - separate the target from the rest of predictors:

In [ ]:
X = data.drop(["age_ret", "mergeid"], axis=1)  # dropping the age of retirement and ID from the data, as it shouldn't have an inherent meaning to the prediction of age.
y = data["age_ret"] # contains only the column of retirement age

X.shape # 212 columns

In [ ]:
X = X.astype("category") # Setting the column type to be "categorical" for all predictor variables

In [ ]:
X['ph012_'] = pd.to_numeric(X['ph012_'], errors='coerce') # Changing back weight column to be numeric

#### 4.2 Split the data into train and test:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)

#### 4.3 Prepare the parameter distribution for the hyperparameter tuning/search:

Only run the code for sections 4.3 - 4.6 if the model and shap files are not loaded (the ones after loading the libraries).

In [ ]:
params = {
    'max_depth': randint(3, 8),
    'learning_rate': uniform(0.005, 0.15),
    'n_estimators': randint(100, 1000),
    'subsample': uniform(0.2, 0.5),
    'colsample_bytree': uniform(0.2, 0.5)
}

#### 4.4 Create a regressor, which will explore parameter values that fit the data the best:

In [ ]:
reg_model = XGBRegressor(
    tree_method = "hist", # Required for categorical support
    enable_categorical = True,       # Enables native categorical handling
    random_state = 42
)

#### 4.5 Perform the Randomized search [run time ~ 25 mins]:

The code below was run once, then it is not needed as we load the model itself.

In [ ]:
random_search = RandomizedSearchCV(
    estimator = reg_model, 
    param_distributions = params, 
    cv = 5, 
    n_iter = 100,
    random_state = 42)

random_search.fit(X_train, y_train)

In [ ]:
# The line below is run only once - after the random search is completed - to save the best model.

#best_model = random_search.best_estimator_

#best_model_2 = random_search.best_estimator_

In [ ]:
# The line below saves the model - only needed to run once.

#joblib.dump(best_model, 'output/best_model.pkl')
#joblib.dump(best_model_2, 'output/best_model_2.pkl') # This is the second model that we use.

#### 4.6 Plotting the mean test score across the different parameter settings/models:

The code below requires the search to be done again. But the plot is already save in the folder.

In [ ]:
sorted_results = pd.DataFrame(random_search.cv_results_).sort_values("rank_test_score") # dataset to store the results

# plot the output of the randomized search:

plt.figure(figsize=(10, 4))
plt.plot(sorted_results["rank_test_score"], 
         sorted_results["mean_test_score"], 
         marker='o',
         markersize=3,
         linewidth=0.5,
         color = "steelblue")
plt.xlabel("Parameter rank")
plt.ylabel("Average test score")
plt.title("Model performance")
plt.grid(True)
#plt.savefig("/Users/justina/Desktop/Data Science/plots/m_performance_par_rank_2.png", dpi=300, bbox_inches='tight')
plt.show()

# The top score is 0.185736.

### 5. Predicting the data - full model with all features in:

In [ ]:
#y_pred = best_model.predict(X_test)
y_pred = best_model_2.predict(X_test)

# Calculates the metrics:

RMSE_full = np.sqrt(mean_squared_error(y_test, y_pred))
R2_full = r2_score(y_test, y_pred)
print("RMSE:", RMSE_full)
print("R²:", R2_full)

#RMSE: 4.813924345110674
#R²: 0.22843654308677708

### 6. Calculate feature importances:

In [ ]:
# Recoding the names of the features:
rename_dict = {
    'dn042_': 'Gender [DN]',
    'it004_': 'Recent internet use [IT]',
    'hc116d3': 'Private long-term health insurance [HC]',
    'country': 'Country',
    'hc116dno': 'Other long-term health insurance [HC]',
    'br039_': 'Alcohol use in last 7 days [BR]',
    'ph048d9': 'Difficulty with heavy-lifting (5kg) [PH]',
    'ph089dno': 'Not bothered by physical weakness [PH]',
    'ph048dno': 'Not difficulties in everyday activities [PH]',
    'ph012_': 'Weight of respondent [PH]',
    'ph084_': 'Trouble with pain [PH]',
    'ph089d4': 'Bothered by fatigue [PH]',
    'hc010_': 'Dentist appointment in last 12 months[HC]',
    'ph006d18': 'Emotional disorders [PH]',
    'hc884_': 'Vaccination for flu [HC]',
    'dn002_': 'Month of birth [DN]', 
    'ex001_': '(Not relevant) [EX]',
    'hc602_': 'Number of doctor visits [HC]', 
    'ex009_': 'Life expectancy [EX]',
    'ph004_': 'Long-term illness [PH]',
    'ph003_': 'Health in general [PH]',
    'br026_': 'Consumption of dairy products per week [BR]',
    'ac012_': 'Life satisfaction [AC]'
}

These two chunks below allow us to compare the feature importances when importances asre based on "gain", and compare how they are based on their "weight":

In [ ]:
# The below exports the gain measure:

booster = best_model_2.get_booster()
gain_score = booster.get_score(importance_type = 'gain')

In [ ]:
gain_df = pd.DataFrame.from_dict(gain_score, orient='index', columns=['Gain'])
gain_df.index.name = 'Feature'
gain_df.reset_index(inplace=True)

gain_df['Feature'] = gain_df['Feature'].replace(rename_dict)

# Sort and take top 10
top_gain_df = gain_df.sort_values(by='Gain', ascending=False).head(15)

# Plot
plt.figure(figsize=(8, 5))
plt.barh(top_gain_df['Feature'], top_gain_df['Gain'], color='lightsteelblue')
plt.xlabel('Gain score')
plt.title('Top 15 features by gain importance (XGBoost)')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.gca().invert_yaxis() 
plt.tight_layout()
#plt.savefig("/Users/justina/Desktop/Data Science/plots/feature_importances_top15_m2_gain.png", dpi=300, bbox_inches='tight')
plt.show()

Now, we calculate weight-based feature importances:

In [ ]:
# Calculates weight-based, scaled feature importances:

importances = best_model_2.feature_importances_

feature_importance_data = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importances
})

feature_importance_data = feature_importance_data.sort_values(by='Importance', ascending=False)

These will be used for extracting the features, and later - plotting them. These turn out to be the same as importances based on "gain".

#### 7.1 Extracting top 15 features [for comparison]:

In [ ]:
# What are the top 15 most important features?

top_15_features = feature_importance_data.head(15)['Feature'].tolist()

# Creating a subset of features for training and testing:

X_train_top15 = X_train[top_15_features]
X_test_top15 = X_test[top_15_features]

# Model fit:

best_model_2.fit(X_train_top15, y_train)

# Predicion:

y_pred_top15 = best_model_2.predict(X_test_top15)

#### 7.2 Extracting top 10 features [for comparison]:

Once the training and testing sets are adjusted, we use the model with the optimal hyperparameters (the one we found above), and train it on the most informative features only.

In [ ]:
best_model_2 = joblib.load('output/best_model_2.pkl') # Loads the model again just in case.

In [ ]:
# What are the top 10 most important features?

top_10_features = feature_importance_data.head(10)['Feature'].tolist()

X_train_top10 = X_train[top_10_features]
X_test_top10 = X_test[top_10_features]

best_model_2.fit(X_train_top10, y_train)

y_pred_top10 = best_model_2.predict(X_test_top10)

#### 7.3 Calculate the metrics:

In [ ]:
RMSE_top_15 = np.sqrt(mean_squared_error(y_test, y_pred_top15))
R2_top_15 = r2_score(y_test, y_pred_top15)
print("RMSE:", RMSE_top_15)
print("R²:", R2_top_15)

In [ ]:
RMSE_top_10 = np.sqrt(mean_squared_error(y_test, y_pred_top10))
R2_top_10 = r2_score(y_test, y_pred_top10)
print("RMSE:", RMSE_top_10)
print("R²:", R2_top_10)

### 8. Plot the model comparison:

In [ ]:
xgb_models = ['Full Model', 'Top 15 Features', 'Top 10 Features']
rmse = [RMSE_full, RMSE_top_15, RMSE_top_10]
r2 = [R2_full, R2_top_15, R2_top_10]

# Set up subplots
fig, ax = plt.subplots(1, 2, figsize=(9, 4.5))

# RMSE plot
bars_rmse = ax[0].bar(xgb_models, rmse, color=['indianred', 'lightsteelblue', 'slategray'])
ax[0].set_title('RMSE Comparison')
ax[0].set_ylabel('RMSE')

# Annotate RMSE bars
for bar in bars_rmse:
    height = bar.get_height()
    ax[0].text(bar.get_x() + bar.get_width() / 2, height + 0.01, f'{height:.2f}', ha='center', va='bottom')
    

# R² plot    
bars_r2 = ax[1].bar(xgb_models, r2, color=['indianred', 'lightsteelblue', 'slategray'])
ax[1].set_title('R² Score Comparison')
ax[1].set_ylabel('R²')

# Annotate R² bars
for bar in bars_r2:
    height = bar.get_height()
    ax[1].text(bar.get_x() + bar.get_width() / 2, height + 0.001, f'{height:.2f}', ha='center', va='bottom')

plt.suptitle('Model performance comparison: feature selection (XGBoost)')
plt.tight_layout()

#plt.savefig("/Users/justina/Desktop/Data Science/plots/m_performance_across_features_xgboost.png", dpi=300, bbox_inches='tight')
plt.show()

### 9. Plot of the top 15 features:

In [ ]:
top_15_features = feature_importance_data.sort_values(by='Importance', ascending=False).head(15)

top_15_features['Feature'] = top_15_features['Feature'].replace(rename_dict)

# Plot
plt.figure(figsize=(8, 5))
plt.barh(top_15_features['Feature'], top_15_features['Importance'], color='lightsteelblue')
plt.xlabel('Importance score')
plt.title('Top 15 feature importances, XGBoost')
plt.gca().invert_yaxis()  # Highest at top
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
#plt.savefig("/Users/justina/Desktop/Data Science/plots/feature_importances_top15.png", dpi=300, bbox_inches='tight')
plt.show()

### 10. SHAP values

In [ ]:
explainer = shap.Explainer(best_model_2)

Kernel tends to crash when running the code below, especially if the explainer is TreeExplainer. We use the simple Explainer instead. The type of the eplainer is "tree" anyways.

In [ ]:
#shap_values_2 = explainer.shap_values(X_test) 
#joblib.dump(shap_values_2, "output/shap_values_2.joblib") #Save

No need to run the code again, the object can be loaded:

In [ ]:
# Load:
shap_values_2 = joblib.load("output/shap_values_2.joblib")

The code below show feature importances based on SHAP values:

In [ ]:
mean_abs_shap = np.abs(shap_values_2).mean(axis=0)
feature_names = X_test.columns

sorted_idx = np.argsort(mean_abs_shap)[::-1]
sorted_names = feature_names[sorted_idx]
sorted_shap = mean_abs_shap[sorted_idx]

clean_names = [rename_dict.get(name, name) for name in sorted_names[:15]]

plt.figure(figsize=(8, 5))
plt.barh(clean_names[:15][::-1], sorted_shap[:15][::-1], color="slategray")
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.xlabel("Mean (SHAP value)")
plt.title("Top 15 feature importances, SHAP")
plt.tight_layout()
#plt.savefig("/Users/justina/Desktop/Data Science/plots/SHAP_top15.png", dpi=300, bbox_inches='tight')
plt.show()

Performing the prediction using 15 and 10 best features based on SHAP values, as a comparison to the ones by XGBoost. Maybe performance with these features is better?

In [ ]:
# What are the top 15 and 10 most important features?

top_10_shap = sorted_names[:10].tolist()

X_train_top10_shap = X_train[top_10_shap] # creating a subset of features for training and testing
X_test_top10_shap = X_test[top_10_shap]

best_model_2 = joblib.load('best_model_2.pkl')

best_model_2.fit(X_train_top10_shap, y_train)

y_pred_top10_shap = best_model_2.predict(X_test_top10_shap)

top_15_shap = sorted_names[:15].tolist()

X_train_top15_shap = X_train[top_15_shap] # creating a subset of features for training and testing
X_test_top15_shap = X_test[top_15_shap]

best_model_2 = joblib.load('best_model_2.pkl') # Just in case we need a new load

best_model_2.fit(X_train_top15_shap, y_train)

y_pred_top15_shap = best_model_2.predict(X_test_top15_shap)


In [ ]:
RMSE_top_15_shap = np.sqrt(mean_squared_error(y_test, y_pred_top15_shap))
R2_top_15_shap = r2_score(y_test, y_pred_top15_shap)

RMSE_top_10_shap = np.sqrt(mean_squared_error(y_test, y_pred_top10_shap))
R2_top_10_shap = r2_score(y_test, y_pred_top10_shap)

In [ ]:
xgboost_shaps = ['Full Model', 'Top 15 Features', 'Top 10 Features']
rmse = [RMSE_full, RMSE_top_15_shap, RMSE_top_10_shap]
r2 = [R2_full, R2_top_15_shap, R2_top_10_shap]

# Set up subplots
fig, ax = plt.subplots(1, 2, figsize=(9, 4.5))

# RMSE plot
bars_rmse = ax[0].bar(xgboost_shaps, rmse, color=['indianred', 'lightsteelblue', 'slategray'])
ax[0].set_title('RMSE Comparison')
ax[0].set_ylabel('RMSE')

# Annotate RMSE bars
for bar in bars_rmse:
    height = bar.get_height()
    ax[0].text(bar.get_x() + bar.get_width() / 2, height + 0.01, f'{height:.2f}', ha='center', va='bottom')

# R² plot
bars_r2 = ax[1].bar(xgboost_shaps, r2, color=['indianred', 'lightsteelblue', 'slategray'])
ax[1].set_title('R² Score Comparison')
ax[1].set_ylabel('R²')

# Annotate R² bars
for bar in bars_r2:
    height = bar.get_height()
    ax[1].text(bar.get_x() + bar.get_width() / 2, height, f'{height:.2f}', ha='center', va='bottom')

plt.suptitle('Model performance comparison: feature selection (SHAP)')
plt.tight_layout()
#plt.savefig("/Users/justina/Desktop/Data Science/plots/m_performance_across_features_shap.png", dpi=300, bbox_inches='tight')
plt.show()


### 11. Interpretation of SHAP

To run the code below, the kernel needs to be restarted, unfortunately. Moreover, make sure to load sections 1-4.2 again, then the 1st and 3rd code-chunk of section 10. There will be less variables in the memory, and the code will run (or should run) without problems.

In [ ]:
explanation = explainer(X_test)

In [ ]:
shap.plots.scatter(explanation[:, "it004_"], color = explanation, show = False, dot_size = 2)
plt.ylabel("SHAP value: recent internet use [IT]")
plt.xlabel("Recent internet use")

cbar_ax = plt.gcf().axes[-1]
cbar_ax.set_ylabel("") 
cbar_ax.set_ylabel("Gender") 
#plt.savefig("/Users/justina/Desktop/Data Science/plots/int_it_gender.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
shap.plots.scatter(explanation[:, "dn042_"], color = explanation, show = False, dot_size = 2)
plt.ylabel("SHAP value: gender")
plt.xlabel("Gender")

cbar_ax = plt.gcf().axes[-1]
cbar_ax.set_ylabel("") 
cbar_ax.set_ylabel("Recent internet use") 
#plt.savefig("/Users/justina/Desktop/Data Science/plots/int_gender_it.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
shap.plots.scatter(explanation[:, "country"], color = explanation, show = False, dot_size = 2)
plt.ylabel("SHAP value: country")
plt.xlabel("Country")

cbar_ax = plt.gcf().axes[-1]
cbar_ax.set_ylabel("") 
cbar_ax.set_ylabel("Gender") 
#plt.savefig("/Users/justina/Desktop/Data Science/plots/int_country_gender.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
shap.plots.scatter(explanation[:, "br039_"], color = explanation, show = False, dot_size = 2)
plt.ylabel("SHAP value: recent alcohol use")
plt.xlabel("Recent alcohol use")

cbar_ax = plt.gcf().axes[-1]
cbar_ax.set_ylabel("") 
cbar_ax.set_ylabel("Gender") 
#plt.savefig("/Users/justina/Desktop/Data Science/plots/int_alcohol_gender.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
shap.plots.scatter(explanation[:, "hc116d3"], color = explanation, show = False, dot_size = 2)
plt.ylabel("SHAP value: any private health insurance?")
plt.xlabel("Private health insurance")

cbar_ax = plt.gcf().axes[-1]
cbar_ax.set_ylabel("") 
cbar_ax.set_ylabel("Gender") 
#plt.savefig("/Users/justina/Desktop/Data Science/plots/int_private_insurance.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
shap.plots.scatter(explanation[:, "hc116dno"], color = explanation, show = False, dot_size = 2)
plt.ylabel("SHAP value: any other health insurance?")
plt.xlabel("Other health insurance")

cbar_ax = plt.gcf().axes[-1]
cbar_ax.set_ylabel("") 
cbar_ax.set_ylabel("Gender") 
#plt.savefig("/Users/justina/Desktop/Data Science/plots/int_other_insurance.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Interesing - but not in the paper.

weight_idx = explanation.feature_names.index("ph012_")

# Create mask for weights > 20
mask = explanation.data[:, weight_idx] > 20

# Filter the explanation object
filtered_explanation = explanation[mask]


shap.plots.scatter(filtered_explanation[:, "ph012_"], color=filtered_explanation, show=False, dot_size=3)
plt.ylabel("SHAP value for respondent's weight")
plt.xlabel("Weight")

cbar_ax = plt.gcf().axes[-1]
cbar_ax.set_ylabel("") 
cbar_ax.set_ylabel("Gender") 
#plt.savefig("/Users/justina/Desktop/Data Science/plots/int_weight_gender.png", dpi=300, bbox_inches='tight')
plt.show()